In [152]:
import collections
import math
import os
import numpy as np

import docker
import fire
import torch

from fedn.utils.helpers.helpers import get_helper, save_metadata, save_metrics

HELPER_MODULE = 'numpyhelper'
helper = get_helper(HELPER_MODULE)

NUM_CLASSES = 10

In [153]:
def _get_data_path():
    """ For test automation using docker-compose. """
    # Figure out FEDn client number from container name
    client = docker.from_env()
    container = client.containers.get(os.environ['HOSTNAME'])
    number = container.name[-1]

    # Return data path
    return f"/var/data/clients/{number}/mnist.pt"


def compile_model():
    """ Compile the pytorch model.

    :return: The compiled model.
    :rtype: torch.nn.Module
    """
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = torch.nn.Linear(784, 64)
            self.fc2 = torch.nn.Linear(64, 32)
            self.fc3 = torch.nn.Linear(32, 10)

        def forward(self, x):
            x = torch.nn.functional.relu(self.fc1(x.reshape(x.size(0), 784)))
            x = torch.nn.functional.dropout(x, p=0.5, training=self.training)
            x = torch.nn.functional.relu(self.fc2(x))
            x = torch.nn.functional.log_softmax(self.fc3(x), dim=1)
            return x

    return Net()


def load_data(data_path, is_train=True):
    """ Load data from disk.

    :param data_path: Path to data file.
    :type data_path: str
    :param is_train: Whether to load training or test data.
    :type is_train: bool
    :return: Tuple of data and labels.
    :rtype: tuple
    """
    if data_path is None:
        data = torch.load(_get_data_path())
    else:
        data = torch.load(data_path)

    if is_train:
        X = data['x_train']
        y = data['y_train']
    else:
        X = data['x_test']
        y = data['y_test']

    # Normalize
    X = X / 255

    return X, y


def save_parameters(model, out_path):
    """ Save model paramters to file.

    :param model: The model to serialize.
    :type model: torch.nn.Module
    :param out_path: The path to save to.
    :type out_path: str
    """
    parameters_np = [val.cpu().numpy() for _, val in model.state_dict().items()]
    helper.save(parameters_np, out_path)


def load_parameters(model_path):
    """ Load model parameters from file and populate model.

    param model_path: The path to load from.
    :type model_path: str
    :return: The loaded model.
    :rtype: torch.nn.Module
    """
    model = compile_model()
    parameters_np = helper.load(model_path)

    params_dict = zip(model.state_dict().keys(), parameters_np)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    return model


def init_seed(out_path='seed.npz'):
    """ Initialize seed model and save it to file.

    :param out_path: The path to save the seed model to.
    :type out_path: str
    """
    # Init and save
    model = compile_model()
    save_parameters(model, out_path)

In [154]:
# def validate(in_model_path, out_json_path, data_path=None, malicious=False, attack=None):
in_model_path = '../trained_model.npz'
out_json_path = None
data_path = '../data/clients/1/mnist.pt'

""" Validate model.

:param in_model_path: The path to the input model.
:type in_model_path: str
:param out_json_path: The path to save the output JSON to.
:type out_json_path: str
:param data_path: The path to the data file.
:type data_path: str
"""
# Load data
x_train, y_train = load_data(data_path)
x_test, y_test = load_data(data_path, is_train=False)

# Load model
model = load_parameters(in_model_path)
model.eval()

# Evaluate
criterion = torch.nn.NLLLoss()
with torch.no_grad():
    train_out = model(x_train)
    training_loss = criterion(train_out, y_train)
    training_accuracy = torch.sum(torch.argmax(
        train_out, dim=1) == y_train) / len(train_out)
    test_out = model(x_test)
    test_loss = criterion(test_out, y_test)
    test_accuracy = torch.sum(torch.argmax(
        test_out, dim=1) == y_test) / len(test_out)

# JSON schema
report = {
    "training_loss": training_loss.item(),
    "training_accuracy": training_accuracy.item(),
    "test_loss": test_loss.item(),
    "test_accuracy": test_accuracy.item(),
}

print(report)
print()
print(train_out)



# validate(in_model_path = '../trained_model.npz', out_json_path = None, data_path = '../data/clients/1/mnist.pt')

{'training_loss': 0.13496573269367218, 'training_accuracy': 0.9609666466712952, 'test_loss': 0.21345260739326477, 'test_accuracy': 0.9327999949455261}

tensor([[-8.1156e+00, -1.4351e+01, -1.2342e+01,  ..., -1.0080e+01,
         -1.0856e+01, -1.0717e+01],
        [-1.7403e-04, -2.8753e+01, -1.1522e+01,  ..., -1.3923e+01,
         -1.5263e+01, -1.5710e+01],
        [-8.6215e+00, -1.0835e+01, -5.8123e+00,  ..., -5.1975e+00,
         -7.4678e+00, -3.7238e+00],
        ...,
        [-8.0171e+00, -1.9583e+01, -6.6697e+00,  ..., -1.5389e+01,
         -1.0198e+01, -1.1493e+01],
        [-1.3804e+01, -1.5508e+01, -1.0357e+01,  ..., -3.5685e-04,
         -1.6709e+01, -9.1669e+00],
        [-1.2159e+01, -8.2342e-02, -3.4008e+00,  ..., -3.5874e+00,
         -4.5685e+00, -8.0196e+00]])


In [155]:
torch.argmax(train_out, dim=1)

tensor([5, 0, 4,  ..., 6, 7, 1])

In [156]:
y_train == 3

tensor([False, False, False,  ..., False, False, False])

In [157]:
np.sum((y_train == 3).tolist())

3073

In [158]:
len(torch.argmax(train_out, dim=1)[y_train == 3])

3073

In [159]:
torch.argmax(train_out, dim=1)[y_train == 3] == 8

tensor([False, False, False,  ..., False, False, False])

In [160]:
np.sum((torch.argmax(train_out, dim=1)[y_train == 3] == 8).tolist())

17

In [161]:
for i in range(10):
    print(f"{i}: {np.sum((torch.argmax(test_out, dim=1)[y_test == 3] == i).tolist())}")

0: 0
1: 1
2: 8
3: 472
4: 0
5: 9
6: 0
7: 8
8: 2
9: 0


In [162]:
np.sum((torch.argmax(test_out, dim=1)[y_test == 3] == 8).tolist()) / len(torch.argmax(test_out, dim=1)[y_test == 3])

0.004